# Environment setting

In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=52e9f557503608de9817b914445e2fc4c1166728f7cc0a51e69caf8bbcb64ac6
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
downloaded = drive.CreateFile({'id':'13RZ0_sSr5HbTm1urv5veWtuRWkWoxbDA'}) # replace the id with id of file you want to access
downloaded.GetContentFile('data.py')

In [5]:
% run data.py

In [6]:
(x_train, y_train), (x_val, y_val), (i2w, w2i), numcls = load_imdb(final=False)

In [7]:
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


#Train,Test data tidyup (Batch size setting, Padding)

In [9]:
batch = []
target = []
for j in range(500):
    tensor_x = []
    for i in range(40):
        tensor_x.append(torch.tensor(x_train[i+j*40], dtype=torch.long))
    tensor_pad = pad_sequence(tensor_x, batch_first=True, padding_value=0)
    batch.append(tensor_pad)
    target.append(torch.tensor(y_train[0+j*40:40+j*40]))

In [10]:
v_batch = []
v_target = []
for j in range(125):
    tensor_x = []
    for i in range(40):
        tensor_x.append(torch.tensor(x_train[i+j*40], dtype=torch.long))
    tensor_pad = pad_sequence(tensor_x, batch_first=True, padding_value=0)
    v_batch.append(tensor_pad)
    v_target.append(torch.tensor(y_train[0+j*40:40+j*40]))

# Model 1: Handmade No-reccurrent NN

In [11]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 16 output channels, 3x3 square convolution
        # kernel
        self.layer1 = nn.Embedding(len(i2w),300)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(300,300)  
        self.fc2 = nn.Linear(300,2)

    def forward(self, x):
        x = self.layer1(x.to(device))
        x = torch.max(F.relu(self.fc1(x.to(device))),1)
        x = self.fc2(x.values)
        return x

net = Net()
print(net)

Net(
  (layer1): Embedding(99430, 300)
  (fc1): Linear(in_features=300, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
)


In [12]:
import torch.optim as optim
optimizer = optim.Adam(net.parameters(),betas=(0.9, 0.999), lr=0.0001)

In [13]:
for epoch in range(1): 
  running_loss = 0.0
  for j in range(500):
    outputs = net.to(device)(batch[j].to(device))
    optimizer.zero_grad()
    loss = torch.nn.functional.cross_entropy(F.softmax(outputs,1),target[j].to(device))
    loss.backward()
    optimizer.step()  

    running_loss += loss.item()
    if j % 50 == 49:    # print every 2000 mini-batches
        print('[%5d] loss: %.3f' %
        (j + 1, running_loss / 50))
        running_loss = 0.0 

print('Finished Training')

[   50] loss: 0.693
[  100] loss: 0.658
[  150] loss: 0.632
[  200] loss: 0.616
[  250] loss: 0.596
[  300] loss: 0.573
[  350] loss: 0.563
[  400] loss: 0.552
[  450] loss: 0.572
[  500] loss: 0.563
Finished Training


In [14]:
correct = 0
total = 0
with torch.no_grad():
    for j in range(125):
        outputs = net(v_batch[j].to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += v_target[j].size(0)
        correct += (predicted == v_target[j].to(device)).sum().item()

print('Accuracy of the network on the 5000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 5000 test images: 80 %


# Model 2: Elman 

In [15]:
class Elman (nn.Module):
    def __init__ (self , insize= 300 , outsize= 300 , hsize= 300 ):
        super (). __init__ ()
        self.lin1 = nn.Linear(insize+hsize,hsize)  
        self.lin2 = nn.Linear(hsize,outsize)
    def forward (self , x, hidden= None ):
        b, t, e = x.size()
        if hidden is None :
            hidden = torch.zeros(b, e, dtype =torch.float)
        outs = []
        
        for i in range (t):
            inp = torch.cat([x[:, i, :], hidden.to(device)], dim = 1)
            inp = F.relu(self.lin1(inp)) 
            out = self.lin2(inp)
            outs.append(out[:, None , :])
        return torch.cat(outs, dim = 1 ), hidden

In [16]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 16 output channels, 3x3 square convolution
        # kernel
        self.layer1 = nn.Embedding(len(i2w),300)
        # an affine operation: y = Wx + b
        self.fc1 = Elman(300,300,300) 
        self.fc2 = nn.Linear(300,2)

    def forward(self, x):
        x = self.layer1(x)
        x = self.fc1(x)
        x = torch.max(F.relu(x[0]),1)
        x = self.fc2(x.values)
        return x

net_Elman = Net()
print(net_Elman)

Net(
  (layer1): Embedding(99430, 300)
  (fc1): Elman(
    (lin1): Linear(in_features=600, out_features=300, bias=True)
    (lin2): Linear(in_features=300, out_features=300, bias=True)
  )
  (fc2): Linear(in_features=300, out_features=2, bias=True)
)


In [17]:
optimizer = optim.Adam(net_Elman.parameters(),betas=(0.9, 0.999), lr=0.001)

In [18]:
for epoch in range(1): 
    running_loss = 0.0 
    for j in range(500):
        outputs = net_Elman.to(device)(batch[j].to(device))
        optimizer.zero_grad()
        loss = torch.nn.functional.cross_entropy(F.softmax(outputs,1),target[j].to(device))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if j % 50 == 49:    # print every 2000 mini-batches
          print('[%5d] loss: %.3f' %
          (j + 1, running_loss / 50))
          running_loss = 0.0 

print('Finished Training')

[   50] loss: 0.597
[  100] loss: 0.476
[  150] loss: 0.488
[  200] loss: 0.460
[  250] loss: 0.448
[  300] loss: 0.446
[  350] loss: 0.502
[  400] loss: 0.481
[  450] loss: 0.507
[  500] loss: 0.474
Finished Training


In [19]:
correct = 0
total = 0
with torch.no_grad():
    for j in range(125):
        outputs = net(v_batch[j].to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += v_target[j].size(0)
        correct += (predicted == v_target[j].to(device)).sum().item()

print('Accuracy of the network on the 5000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 5000 test images: 80 %


# Model 3: Recurrent NN

In [20]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 16 output channels, 3x3 square convolution
        # kernel
        self.layer1 = nn.Embedding(len(i2w),300)
        # an affine operation: y = Wx + b
        self.fc1 = nn.RNN(300,300) 
        self.fc2 = nn.Linear(300,2)

    def forward(self, x):
        x = self.layer1(x)
        x = self.fc1(x)
        x = torch.max(F.relu(x[0]),1)
        x = self.fc2(x.values)
        return x

net_Rnn = Net()
print(net_Rnn)

Net(
  (layer1): Embedding(99430, 300)
  (fc1): RNN(300, 300)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
)


In [21]:
optimizer = optim.Adam(net_Rnn.parameters(),betas=(0.9, 0.999), lr=0.001)

In [22]:
for epoch in range(1):
    running_loss = 0.0 
    for j in range(500):
        outputs = net_Rnn.to(device)(batch[j].to(device))
        optimizer.zero_grad()
        loss = torch.nn.functional.cross_entropy(F.softmax(outputs,1),target[j].to(device))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if j % 50 == 49:    
          print('[%5d] loss: %.3f' %
          (j + 1, running_loss / 50))
          running_loss = 0.0 

print('Finished Training')

[   50] loss: 0.691
[  100] loss: 0.644
[  150] loss: 0.581
[  200] loss: 0.535
[  250] loss: 0.502
[  300] loss: 0.467
[  350] loss: 0.500
[  400] loss: 0.531
[  450] loss: 0.538
[  500] loss: 0.531
Finished Training


In [24]:
correct = 0
total = 0
with torch.no_grad():
    for j in range(125):
        outputs = net_Rnn(v_batch[j].to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += v_target[j].size(0)
        correct += (predicted == v_target[j].to(device)).sum().item()

print('Accuracy of the network on the 5000 test: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 5000 test: 84 %


# Model 4: LSTM

In [25]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 16 output channels, 3x3 square convolution
        # kernel
        self.layer1 = nn.Embedding(len(i2w),300)
        # an affine operation: y = Wx + b
        self.fc1 = nn.LSTM(300,300)  
        self.fc2 = nn.Linear(300,2)

    def forward(self, x):
        x = self.layer1(x)
        x = self.fc1(x)
        x = torch.max(F.relu(x[0]),1)
        x = self.fc2(x.values)
        return x

net_LSTM = Net()
print(net_LSTM)

Net(
  (layer1): Embedding(99430, 300)
  (fc1): LSTM(300, 300)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
)


In [26]:
optimizer = optim.Adam(net_LSTM.parameters(),betas=(0.9, 0.999), lr=0.001)

In [27]:
for epoch in range(1): 
    running_loss = 0.0
    for j in range(500):
        outputs = net_LSTM.to(device)(batch[j].to(device))
        optimizer.zero_grad()
        loss = torch.nn.functional.cross_entropy(F.softmax(outputs,1),target[j].to(device))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if j % 50 == 49:    # print every 2000 mini-batches
          print('[%5d] loss: %.3f' %
          (j + 1, running_loss / 50))
          running_loss = 0.0 

print('Finished Training')

[   50] loss: 0.647
[  100] loss: 0.530
[  150] loss: 0.532
[  200] loss: 0.502
[  250] loss: 0.489
[  300] loss: 0.480
[  350] loss: 0.499
[  400] loss: 0.519
[  450] loss: 0.560
[  500] loss: 0.579
Finished Training


In [28]:
correct = 0
total = 0
with torch.no_grad():
    for j in range(125):
        outputs = net_LSTM(v_batch[j].to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += v_target[j].size(0)
        correct += (predicted == v_target[j].to(device)).sum().item()

print('Accuracy of the network on the 5000 test: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 5000 test: 83 %
